In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read the spotify data in from data.csv
url = 'https://media.githubusercontent.com/media/jossharlequin/spotify-popularity-project/main/Resources/data.csv'
df = pd.read_csv(url)
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [12]:
# Drop unused columns
spotify_df = df.drop(columns=['artists', 'name', 'id', 'release_date', 'year'])
spotify_df['seconds'] = spotify_df['duration_ms']/1000
spotify_df = df.drop(columns=['artists', 'name', 'id', 'release_date', 'year'])
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169909 entries, 0 to 169908
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      169909 non-null  float64
 1   danceability      169909 non-null  float64
 2   duration_ms       169909 non-null  int64  
 3   energy            169909 non-null  float64
 4   explicit          169909 non-null  int64  
 5   instrumentalness  169909 non-null  float64
 6   key               169909 non-null  int64  
 7   liveness          169909 non-null  float64
 8   loudness          169909 non-null  float64
 9   mode              169909 non-null  int64  
 10  popularity        169909 non-null  int64  
 11  speechiness       169909 non-null  float64
 12  tempo             169909 non-null  float64
 13  valence           169909 non-null  float64
 14  seconds           169909 non-null  float64
dtypes: float64(10), int64(5)
memory usage: 19.4 MB


In [48]:
# Setting popularity as the target variable and setting the remaining columns as features
y = spotify_df.popularity.values
X = spotify_df.drop(columns='popularity').values

Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity',
       'speechiness', 'tempo', 'valence', 'seconds'],
      dtype='object')


In [5]:
# Scaling the data using StandarScaler as a preprocessing step for the neural network
# Creating the StandardScalar instance
scaler = StandardScaler()

# Fitting the X data
X_scaler = scaler.fit(X)

# Scaling the X data
X_scaled = X_scaler.transform(X)

# Splitting training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

In [21]:
# Binning the target variable into 10 groups  
bins = [-1, 20, 40, 60, 80, 100]
labels = [0, 1, 2, 3, 4]
y_train_binned = pd.cut(y_train, bins=bins, labels=labels)
y_test_binned = pd.cut(y_test, bins=bins, labels=labels)

encoder = OneHotEncoder(sparse=False)
y_train_one_hot = encoder.fit_transform(y_train_binned.reshape(-1,1))
y_test_one_hot = encoder.transform(y_test_binned.reshape(-1,1))

c:\Users\josep\anaconda3\envs\dev\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [23]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=14))
nn_model.add(tf.keras.layers.Dense(5, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=10)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3983/3983 [==============================] - 8s 2ms/step - loss: 1.0009 - accuracy: 0.5714
Epoch 2/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9323 - accuracy: 0.6010
Epoch 3/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9242 - accuracy: 0.6039
Epoch 4/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0.9196 - accuracy: 0.6047
Epoch 5/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0.9171 - accuracy: 0.6061
Epoch 6/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9149 - accuracy: 0.6061
Epoch 7/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9133 - accuracy: 0.6072
Epoch 8/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9120 - accuracy: 0.6063
Epoch 9/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9109 - accuracy: 0.6060
Epoch 10/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.910

In [24]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="tanh", input_dim=14))
nn_model.add(tf.keras.layers.Dense(5, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=10)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy score: {model_accuracy}")

Epoch 1/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9834 - accuracy: 0.5837
Epoch 2/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9310 - accuracy: 0.6026
Epoch 3/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9232 - accuracy: 0.6038
Epoch 4/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9176 - accuracy: 0.6056
Epoch 5/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9138 - accuracy: 0.6065
Epoch 6/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9115 - accuracy: 0.6070
Epoch 7/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9098 - accuracy: 0.6075
Epoch 8/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9085 - accuracy: 0.6074
Epoch 9/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.9075 - accuracy: 0.6077
Epoch 10/10
3983/3983 [==============================] - 7s 2ms/step - loss: 0.907

In [59]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=50, activation="tanh", input_dim=14))
nn_model.add(tf.keras.layers.Dense(units=50, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=5, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=10)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9471 - accuracy: 0.5959
Epoch 2/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9127 - accuracy: 0.6061
Epoch 3/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.9034 - accuracy: 0.6083
Epoch 4/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8983 - accuracy: 0.6101
Epoch 5/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8948 - accuracy: 0.6113
Epoch 6/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8916 - accuracy: 0.6115
Epoch 7/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0.8891 - accuracy: 0.6123
Epoch 8/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8874 - accuracy: 0.6129
Epoch 9/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8854 - accuracy: 0.6148
Epoch 10/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.884

In [61]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=50, activation="tanh", input_dim=14))
nn_model.add(tf.keras.layers.Dense(units=50, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=50, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=5, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=10)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3983/3983 [==============================] - 10s 2ms/step - loss: 0.9388 - accuracy: 0.5977
Epoch 2/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0.9106 - accuracy: 0.6058
Epoch 3/10
3983/3983 [==============================] - 10s 3ms/step - loss: 0.9022 - accuracy: 0.6089
Epoch 4/10
3983/3983 [==============================] - 10s 3ms/step - loss: 0.8969 - accuracy: 0.6102
Epoch 5/10
3983/3983 [==============================] - 11s 3ms/step - loss: 0.8928 - accuracy: 0.6123
Epoch 6/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8894 - accuracy: 0.6128
Epoch 7/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8870 - accuracy: 0.6132
Epoch 8/10
3983/3983 [==============================] - 8s 2ms/step - loss: 0.8846 - accuracy: 0.6138
Epoch 9/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0.8831 - accuracy: 0.6156
Epoch 10/10
3983/3983 [==============================] - 9s 2ms/step - loss: 0